In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from helper import ExecuteScript, AddRecords, TimeInstance, CreateEngine

In [2]:
# User customizable stuff
server = "mssql.ad.tdt"
database = "sqlHeapTest"
numRecords = 400000
results = []

# Setup database for testing
Connect to the base, and run the setupDB.sql contents

In [3]:
engine = CreateEngine(server)

In [4]:
ExecuteScript(engine, "setupDB.sql")

# Setup Fixture Data
Using the newly created setup, connect and insert some test records

In [5]:
engine = CreateEngine(server, database)
with engine.connect() as conn:
    AddRecords(numRecords, conn)

/var/folders/d7/4cl_f7wd7y5_c9gcsjbt78y80000gn/T/ipykernel_98160/2762592990.py:1: SAWarning: No driver name specified; this is expected by PyODBC when using DSN-less connections
  engine = sa.create_engine(f"mssql+pyodbc://{server}/{database}?Driver=ODBC+Driver+18+for+SQL+Server&Trusted_Connection=yes&TrustServerCertificate=Yes")


# Test 1
Setup a clustered index, timing how long it takes on one column

In [ ]:
t = TimeInstance(lambda: ExecuteScript(engine, "test1.sql"))

results.append({
    "Test": "Test 1",
    "Description": "Create clustered Index, one column",
    "Time (s)": t
})

# Test 2
Remove the clustered index, timing how long it takes

In [ ]:
t = TimeInstance(lambda: ExecuteScript(engine, "test2.sql"))
results.append({
    "Test": "Test 2",
    "Description": "Drop single-column Clustered Index",
    "Time (s)": t
})

# Test 3
Insert another bunch of records and re-add the clustered index

In [ ]:
with engine.connect() as conn:
    AddRecords(numRecords, conn)

In [ ]:
t = TimeInstance(lambda: ExecuteScript(engine, "test1.sql"))
results.append({
    "Test": "Test 3",
    "Description": "Create clustered Index, one column, extra records",
    "Time (s)": t
})

# Test 4
Drop the clustered index, again timing

In [ ]:
t = TimeInstance(lambda: ExecuteScript(engine, "test2.sql"))
results.append({
    "Test": "Test 4",
    "Description": "Drop single-column Clustered Index, extra records",
    "Time (s)": t
})

# Second Test!  With 2 columns in index

# Prep Database

In [ ]:
engine = CreateEngine(server)
ExecuteScript(engine, "setupDB.sql")

In [ ]:
engine = CreateEngine(server, database)

with engine.connect() as conn:
    AddRecords(numRecords, conn)

# Test 5
Setup a clustered index, two columns, seeing how long it takes

In [ ]:
t = TimeInstance(lambda: ExecuteScript(engine, "test3.sql"))
results.append({
    "Test": "Test 5",
    "Description": "Create clustered Index, two columns",
    "Time (s)": t
})

# Test 6
Remove the clustered index, timing how long it takes

In [ ]:
t = TimeInstance(lambda: ExecuteScript(engine, "test2.sql"))
results.append({
    "Test": "Test 6",
    "Description": "Drop dual-column Clustered Index",
    "Time (s)": t
})

# Test 7
Insert another bunch of records and re-add the clustered index

In [ ]:
with engine.connect() as conn:
    AddRecords(numRecords, conn)

In [ ]:
t = TimeInstance(lambda: ExecuteScript(engine, "test3.sql"))
results.append({
    "Test": "Test 7",
    "Description": "Create clustered Index, 2 columns, extra records",
    "Time (s)": t
})

# Test 8
Drop the clustered Index, again timing

In [ ]:
t = TimeInstance(lambda: ExecuteScript(engine, "test2.sql"))
results.append({
    "Test": "Test 8",
    "Description": "Drop dual-column Clustered Index, extra records",
    "Time (s)": t
})

# Plot Results

In [ ]:
pdResults = pd.DataFrame.from_dict(results)
pdResults

In [ ]:
pdResults["Time (s)"] = pdResults["Time (s)"].astype(float)

In [ ]:
plt.figure(figsize=(15,8))
sns.barplot(data=pdResults, x="Test", y="Time (s)")